In [1]:
# will autoupdate any of the packages imported:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
# import pyclifford
from pyclifford import utils, paulialg, stabilizer
from pyclifford.utils import acq, ipow, clifford_rotate, pauli_transform
from pyclifford.paulialg import Pauli, PauliList, PauliPolynomial

In [3]:
def decompose(g, gs, ps):
    '''  Decompose a pauli string to phase*destabilizers*stabilizers
    Parameters:
    g: int(2*N) - the binary vector of a pauli string
    gs: int(2*N,2*N) - the full tableau
    ps: int(2*N) - phase of stabilizer and destabilizer
    
    Returns:
    phase: int - phase in terms of imaginery power
    b: int(N) - binary encoding of destabilizer decomposition
    c: int(N) - binary encoding of stabilizer decomposition
    '''
    phase = 0
    tmp_p = np.zeros_like(g)
    N = gs.shape[0]//2
    b = np.zeros(N).astype(int)
    c = np.zeros(N).astype(int)
    for i in range(N):
        if acq(g,gs[i]): #anti-commute
            b[i] = 1
            phase = phase - ipow(tmp_p,gs[i+N]) + ps[i+N]
            tmp_p = (tmp_p+gs[i+N])%2
    for i in range(N):
        if acq(g,gs[i+N]): #anti-commute
            c[i] = 1
            phase = phase - ipow(tmp_p,gs[i]) + ps[i]
            tmp_p = (tmp_p+gs[i])%2
    return phase%4, tmp_p, b, c

The above function will decompose a pauli string into combination of destabilizer generators and stabilizer generators.

Let's see an example, if the stabilizer generators are $g_0=-ZZ$ and $g_1=XX$ and destabilizer generators are $d_0=IX$, $d_1=XI$.

In [4]:
gs = stabilizer.stabilizer_state("-ZZ","XX").gs
ps = stabilizer.stabilizer_state("-ZZ","XX").ps

In [5]:
gs

array([[0, 1, 0, 1],
       [1, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0]])

In [6]:
ps

array([2, 0, 0, 0])

In [7]:
decompose(np.array([1,0,1,1]),gs,ps)

(1, array([1, 0, 1, 1]), array([0, 1]), array([1, 1]))

From the result, we see pauli string $XY$ can be decomposed as
$$XY = i^1 d_0^0 d_1^1 g_0^1 g_1^1=i(XI)(-ZZ)(XX)$$

In [8]:
# def list2num(lst):
#     return np.sum(2**np.arange(len(lst)) * np.array(list(reversed(lst))))

# def num2list(num, N):
#     assert num < 2**N
#     return np.array(list(map(int, bin(num)[2:].zfill(N))))

In [9]:
class GeneralStabilizerState(object):
    '''Represents a stabilizer state. This is a subclass of PauliList.
        rho = 1/2^r prod_{a=1}^{N-r} (1+ Pauli[g_a,p_a])/2

    The stabilizer state is specified by a stablizer tableau, as a binary matrix
    of the shape (2*N, 2*N).
        rows [0,r) - standby stabilizers
        rows [r,N) - active stabilizers (g_a)
        rows [N,N+r) - standby destabilizers
        rows [N+r,2*N) - active destabilizers
    The stabilizers and destablizers in the tableau forms a list of Pauli 
    operators, which can be represented as a subclass of PauliList.

    Parameters:
    gs: int (2*N, 2*N) - strings of Pauli operators in the stabilizer tableau.
    ps: int (2*N) - phase indicators (should only be 0 or 2).
    r:  int  - number of logical qubits (log2 rank of density matrix)
    
    Note:
    Current version only implements pure stabilizer state (r=0).
    '''
    # def __init__(self, *args, **kwargs):
    def __init__(self, chi, gs, ps):
        self.chi = chi
        self.gs = gs
        self.ps = ps
        
    # def __repr__(self):
    #     ''' will only show active stabilizers, 
    #         to see the full stabilizer tableau, convert to PauliList by [:] '''
    #     subrepr = repr(self.stabilizers)
    #     if subrepr == '':
    #         return 'StabilizerState()'
    #     else:
    #         return 'StabilizerState(\n{})'.format(subrepr).replace('\n','\n  ')

    @property
    def stabilizers(self):
        return self[self.r:self.N]
    
    def copy(self):
        return self.chi.copy(), self.gs.copy(), self.ps.copy()

    def rotate_by(self, generator, mask=None):
        # perform Clifford transformation by Pauli generator (in-place)
        if mask is None:
            clifford_rotate(generator.g, generator.p, self.gs, self.ps)
        else:
            mask2 = numpy.repeat(mask,  2)
            self.gs[:,mask2], self.ps = clifford_rotate(
                generator.g, generator.p, self.gs[:,mask2], self.ps)
        return self
        
    def transform_by(self, generator, mask=None):
        # perform Clifford transformation by Clifford map (in-place)
        if mask is None:
            self.gs, self.ps = pauli_transform(self.gs, self.ps, 
                clifford_map.gs, clifford_map.ps)
        else:
            mask2 = numpy.repeat(mask, 2)
            self.gs[:,mask2], self.ps = pauli_transform(
                self.gs[:,mask2], self.ps, clifford_map.gs, clifford_map.ps)
        return self
    
    def pauli_chnl_evol(self, phi, pl, pr):
        '''Perform general Clifford channel evolution.
        
        Parameters:
        phi: [c1, c2, ...] - list of Pauli channel coefficients.
        pl:  [int (2*N), int (2*N), ...] - list of left-multiplying Pauli ops.
        pr:  [int (2*N), int (2*N), ...] - list of right-multiplying Pauli ops. 
        
        Returns:
        chi in-place modified.
        '''
        # pre-store alpha, b, c to avoid redundant call of decompose
        al, bl, cl = [], [], []
        ar, br, cr = [], [], []
        for pm, pn in zip(pl, pr):
            am, _, bm, cm = decompose(pm, self.gs, self.ps) 
            an, _, bn, cn = decompose(pn, self.gs, self.ps)
            al.append(am)
            bl.append(bm)
            cl.append(cm)
            ar.append(an)
            br.append(bn)
            cr.append(cn)
        # update chi
        chinew = {}
        for (i, j), chiij in self.chi.items():
            i, j = np.array(list(i)), np.array(list(j))
            for idx in range(len(phi)):
                phimn = phi[idx]
                am, bm, cm = al[idx], bl[idx], cl[idx]
                an, bn, cn = ar[idx], br[idx], cr[idx]
                # update i, j
                i = (i + bm)%2
                j = (j + bn)%2
                # update chij
                ipow = (am-an+2*(i@cm+j@cn))%4
                chiij *= phimn * 1j**ipow
            keynew = (tuple(i), tuple(j))
            if keynew in chinew:
                chinew[keynew] += chiij
            else:
                chinew[key] = chiij
        self.chi = chinew
        
    def pauli_expect(self, obs):
        '''Evaluate expectation values of Pauli observables on the
generalized stabilizer state.
        
        Parameters:
        obs: observable, can be Pauli, PauliPolynomial, PauliList
        z: fugacity of operator weight
        
        Returns:
        out: output (depending on the type of obs)
            * Pauli: promote to PauliPolynomial
            * PauliPolynomial O: Tr(rho O z^|O|)
            * Paulilist [O_i]: [Tr(rho O_i z^|O_i|)]
        '''
        if isinstance(obs, Pauli):
            # cast Pauli to PauliPolynomial
            return self.pauli_expect(obs.as_polynomial())
        elif isinstance(obs, PauliPolynomial):
            # cast PauliPolynomial to PauliList
            xs = self.expect(PauliList(obs.gs, obs.ps))
            return numpy.sum(obs.cs * xs)
        elif isinstance(obs, PauliList):
            (L, Ng) = obs.gs.shape
            N = Ng//2
            xs = np.zeros(L, dtype=np.complex_) # expectation values
            pa = 0
            for k in range(L):
                a, _, b, c = decompose(obs.gs[k], self.gs, self.ps)
                for (i, j), chi in self.chi.items():
                    i, j = np.array(list(i)), np.array(list(j))
                    # print(f'i: {i.shape}, j: {j.shape}, b:{b.shape}')
                if np.all((i+j+b) == 0):
                    ipow = (a + obs.ps[k] + 2*i@c) % 4
                    xs[k] += chiij * 1j**ipow
            return xs 

### Test usage of generalized stabilizer

In [10]:
# creat generalized stabilizer
h1 = (0, 1)
h2 = (1, 0)
key = (h1, h2)
ele = 0.125
chi = {key: ele}

ss = stabilizer.stabilizer_state("-ZZ","XX")
gs = ss.gs
ps = ss.ps
gss = GeneralStabilizerState(chi, gs, ps)
# (_, N2) = gs.shape
# N = N2//2
# ds = gs[N:2*N]

In [11]:
gss.chi

{((0, 1), (1, 0)): 0.125}

In [12]:
# test Clifford rotation

In [13]:
# test Pauli channel evolution
phi = np.array([0.25, 0.25, 0.25, 0.25], dtype=np.complex_)
pl = [np.array([0,0,0,1]),np.array([0,0,1,0]),np.array([0,1,0,0]),np.array([1,0,0,0])]
pr = [np.array([1,0,0,0]),np.array([0,1,0,0]),np.array([0,0,1,0]),np.array([0,0,0,1])]
 
gss.pauli_chnl_evol(phi, pl, pr)

In [14]:
gss.chi

{((0, 1), (1, 0)): (0.00048828125-0j)}

In [15]:
testList = np.zeros(4)

In [16]:
testList

array([0., 0., 0., 0.])

In [18]:
# test Pauli expectation
h1 = (0, 0)
h2 = (0, 0)
chi = {(h1, h2): 1}

ss = stabilizer.stabilizer_state("-ZZ","XX")
gss = GeneralStabilizerState(chi, ss.gs, ss.ps)

In [19]:
ss.expect(paulialg.paulis('IZ','-YX'))

array([0, 0])

In [20]:
gss.pauli_expect(paulialg.paulis('IZ','-YX'))

array([0.+0.j, 0.+0.j])